# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [25]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [26]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [27]:
#Install the dependencies
!pip install "dlt[duckdb]"

### Question 1: What is the sum of the outputs of the generator for limit = 5?

In [28]:
sum_gen = sum(square_root_generator(5))
print(sum_gen)

8.382332347441762


### Question 2: What is the 13th number yielded by the generator?

In [29]:
gen13 = square_root_generator(13)
index = 0
for item in gen13:
    index += 1
    if index == 13:
        print(item)

3.605551275463989


### Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [30]:
import duckdb
import dlt

# Define the generators
generator1 = people_1()
generator2 = people_2()

# Create a pipeline to load the data
generator_pipelines = dlt.pipeline(
    destination='duckdb',
    dataset_name='people',
)

# define the connection to the database
connection = duckdb.connect(f'{generator_pipelines.pipeline_name}.duckdb')

# Load the first generator
run1 = generator_pipelines.run(
    data=generator1,
    table_name='people',
    write_disposition='replace'
)

# Load the second generator
run2 = generator_pipelines.run(
    data=generator2,
    table_name='people',
    write_disposition='append'
)

# check the statuses
print(run1)
print('--')
print(run2)

# take a look at the data
print('--')
print('Loaded Table:')
tables = connection.sql('SELECT * FROM people.people').df()
display(tables)

# ages of all people
print('Sum of ages of all people after appending the 2nd generator:')
ages = connection.sql('SELECT SUM(Age) FROM people.people').df()
display(ages)


Pipeline dlt_ipykernel_launcher load step completed in 0.13 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/dimi/Developer/Code/Data Science/Data Engineering ZoomCamp/workshops/01 data ingestion with dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707597068.7743511 is LOADED and contains no failed jobs
--
Pipeline dlt_ipykernel_launcher load step completed in 0.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/dimi/Developer/Code/Data Science/Data Engineering ZoomCamp/workshops/01 data ingestion with dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707597069.251689 is LOADED and contains no failed jobs
--
Loaded Table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707597068.7743511,JgWDJDrbak3tJA,None
1,2,Person_2,27,City_A,1707597068.7743511,Y7jQkcRwnSLQkg,None
2,3,Person_3,28,City_A,1707597068.7743511,Ps+AJE46lo6lzg,None
3,4,Person_4,29,City_A,1707597068.7743511,B2MotKswlptYYQ,None
4,5,Person_5,30,City_A,1707597068.7743511,5cYLoxuxXLCX3Q,None
5,3,Person_3,33,City_B,1707597069.251689,taBSOSLLTfBDXQ,Job_3
6,4,Person_4,34,City_B,1707597069.251689,TdnzMlVwWo7yRg,Job_4
7,5,Person_5,35,City_B,1707597069.251689,sktelTmt9PzcZw,Job_5
8,6,Person_6,36,City_B,1707597069.251689,LVJKE3RCMKSERA,Job_6
9,7,Person_7,37,City_B,1707597069.251689,BGUzEat8kU4L2A,Job_7


Sum of ages of all people after appending the 2nd generator:


,sum(Age)
0,353.0


### Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [31]:
import duckdb
import dlt

# Define the generators
generator1 = people_1()
generator2 = people_2()

# Create a pipeline to load the data
generator_pipelines = dlt.pipeline(
    destination='duckdb',
    dataset_name='people',
)

# define the connection to the database
connection = duckdb.connect(f'{generator_pipelines.pipeline_name}.duckdb')

# Load the first generator
run1 = generator_pipelines.run(
    data=generator1,
    table_name='people',
    write_disposition='replace'
)


## find out what this shit does
connection.sql(f'SET search_path = {generator_pipelines.pipeline_name}')

# Load the second generator
run2 = generator_pipelines.run(
    data=generator2,
    table_name='people',
    write_disposition='merge',
    primary_key='id'
)

# check the statuses
print(run1)
print('--')
print(run2)

# take a look at the data
print('--')
print('Loaded Table:')
tables = connection.sql('SELECT * FROM people.people').df()
display(tables)

# ages of all people
print('Sum of ages of all people after merging the 1st and 2nd generator:')
ages = connection.sql('SELECT SUM(Age) FROM people.people').df()
display(ages)

Pipeline dlt_ipykernel_launcher load step completed in 0.13 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/dimi/Developer/Code/Data Science/Data Engineering ZoomCamp/workshops/01 data ingestion with dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707597069.779829 is LOADED and contains no failed jobs
--
Pipeline dlt_ipykernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/dimi/Developer/Code/Data Science/Data Engineering ZoomCamp/workshops/01 data ingestion with dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707597070.2653718 is LOADED and contains no failed jobs
--
Loaded Table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707597069.779829,iv1HntwmuAVLGA,None
1,2,Person_2,27,City_A,1707597069.779829,rahpM+fVZcSt0Q,None
2,5,Person_5,35,City_B,1707597070.2653718,gRuqAemqtEYDLA,Job_5
3,3,Person_3,33,City_B,1707597070.2653718,VzUpmzbt+nOHmg,Job_3
4,6,Person_6,36,City_B,1707597070.2653718,NM060CaBQfT27Q,Job_6
5,8,Person_8,38,City_B,1707597070.2653718,zYv8SpvFaN+RJA,Job_8
6,4,Person_4,34,City_B,1707597070.2653718,QTge6DDhnI6PWw,Job_4
7,7,Person_7,37,City_B,1707597070.2653718,x7ftXBCyMUTB1Q,Job_7


Sum of ages of all people after merging the 1st and 2nd generator:


,sum(Age)
0,266.0
